Notebook to generate ROIs from Mouse ABA atlas
 - required files include `TEMPLATE_LABEL_NIFTI` and `json_file`
 - outputs ROIs as niftis to the `roi_dir`

In [2]:
from typing import Dict, List, Tuple
from pathlib import Path
import nibabel as nib
import numpy as np

mouse_template_dir = Path("/opt/animalfmritools/animalfmritools/data_template/MouseABA")
TEMPLATE_LABEL_NIFTI = mouse_template_dir / "P56_Annotation_downsample2.nii.gz"
json_file = mouse_template_dir / "ABA_ontology.json"
roi_dir = mouse_template_dir / "rois"
if not roi_dir.exists():
    roi_dir.mkdir()

Functions

In [3]:
def load_json(json_path: Path) -> Dict:
    import json
    
    with open(json_path, 'r') as f:
        data = json.load(f)

    return data

def assert_keys(check_dict: Dict, list_keys: List[str]) -> None:

    for k in check_dict.keys():
        assert k in list_keys

def organize_substructures(aba_dict: Dict, target_key: str, expected_keys: List[str]) -> Dict:
    sub_dict = organize_main_structures(aba_dict)[target_key]['children']
    organized_dict = {item['name']: item for item in sub_dict}
    assert_keys(organized_dict, expected_keys)
    return organized_dict

EXPECTED_KEYS_MAIN = [
    'Basic cell groups and regions',
    'fiber tracts',
    'ventricular systems',
    'grooves',
    'retina'
]

EXPECTED_KEYS_GM = [
    "Cerebrum",
    "Brain stem",
    "Cerebellum",
]

EXPECTED_KEYS_WM = [
    "cranial nerves",
    "cerebellum related fiber tracts",
    "supra-callosal cerebral white matter",
    "lateral forebrain bundle system",
    "extrapyramidal fiber systems",
    "medial forebrain bundle system",
]

EXPECTED_KEYS_VS = [
    "lateral ventricle",
    "interventricular foramen",
    "third ventricle",
    "cerebral aqueduct",
    "fourth ventricle",
    "central canal, spinal cord/medulla",
]

def organize_main_structures(aba_dict: Dict) -> Dict:
    org = aba_dict['msg'][0]['children']
    organized_dict = {item['name']: item for item in org}
    assert_keys(organized_dict, EXPECTED_KEYS_MAIN)
    return organized_dict

def organize_gm_structures(aba_dict: Dict) -> Dict:
    return organize_substructures(aba_dict, 'Basic cell groups and regions', EXPECTED_KEYS_GM)

def organize_wm_structures(aba_dict: Dict) -> Dict:
    return organize_substructures(aba_dict, 'fiber tracts', EXPECTED_KEYS_WM)

def organize_vs_structures(aba_dict: Dict) -> Dict:
    return organize_substructures(aba_dict, 'ventricular systems', EXPECTED_KEYS_VS)

def extract_levels(
    children: Dict, 
    extract_level: int,
    level: int = 1, 
    graph_idxs: Dict = {},
    main_key = None,
    verbose = False,
):

    from copy import deepcopy

    graph_idxs = deepcopy(graph_idxs)
    
    for child in children:
        k_tuple = (child['name'], child['acronym'])
        v_tuple = (child['name'], child['acronym'], child['graph_order'], child['color_hex_triplet'], child['st_level'], child['ontology_id'])
        if extract_level == level:
            graph_idxs[k_tuple] = [v_tuple]
            main_key = k_tuple
            tracker = 'x'
        elif level > extract_level:
            graph_idxs[main_key].append(v_tuple)
            tracker = '>'
        else:
            tracker = 'o'

        if verbose:
            print(f"{tracker} [{str(child['graph_order']).zfill(4)}] {'-'*level} {child['name']} [{child['acronym']}] {level}")
            
        if 'children' in child:
            graph_idxs = extract_levels(child['children'], extract_level, level + 1, graph_idxs, main_key, verbose)

    return graph_idxs

def check_template_for_idx(template_idx: int, template_nifti: str = TEMPLATE_LABEL_NIFTI) -> bool:
    data = nib.load(template_nifti).get_fdata()
    n_voxels_with_idx = np.where(data==template_idx)[0].shape[0]
    if n_voxels_with_idx > 0:
        return True
    else:
        return False

def get_template_coords_from_idx(template_idx: int, template_nifti: str = TEMPLATE_LABEL_NIFTI) -> Tuple:
    data = nib.load(template_nifti).get_fdata()
    coords = np.where(data == template_idx)

    return coords

def save_template_roi(
    parent_structure_label: str,
    roi_label: str,
    template_coords: Tuple, 
    outdir: Path,
    template_nifti: str = TEMPLATE_LABEL_NIFTI
) -> None:

    
    template_img = nib.load(template_nifti)
    template_data = template_img.get_fdata()
    roi_data = np.zeros(template_data.shape)
    roi_data[(template_coords[0,:], template_coords[1,:], template_coords[2,:])] = 1
    roi_img = nib.Nifti1Image(
        roi_data, 
        header = template_img.header,
        affine = template_img.affine,
    )
    output_path = outdir / f"P56_desc-{parent_structure_label}_roi-{roi_label}.nii.gz"
    if not Path(output_path).exists():
        print(f"Saving to {output_path}.")
        nib.save(roi_img, output_path)
    else:
        print(f"{output_path} already exists.")
    
def parse_children(
    children, 
    level = 1, 
    parent_structure = None, 
    previous_structure_label = None, 
    previous_structure_name = None,
    structure_mapping = None, 
    roi_hierarchy = None,
    outdir: Path = None
):
    
    if structure_mapping is None:
        structure_mapping = dict()

    if roi_hierarchy is None:
        roi_hierarchy = dict()

    if len(children) > 0:
        #print('ROI includes: ')
        exist_first = False
        create_roi  = False
        for c in children:

            # Fill structure mapping
            structure_mapping[c['acronym']] = c['name']

            # Fill roi hierarchy mapping
            if previous_structure_label is not None:
                if previous_structure_label not in roi_hierarchy.keys():
                    roi_hierarchy[previous_structure_label] = [c['acronym']]
                else:
                    roi_hierarchy[previous_structure_label].append(c['acronym'])

            # Check if nifti label exists in the atlas
            nifti_label = c['graph_order']
            label_exists = check_template_for_idx(nifti_label)
            # Print info
            if label_exists:
                create_roi = True
                # print(f"{'-'*level} [{c['acronym']}] {c['name']} {c['color_hex_triplet']} {label_exists} || PRIOR: [{previous_structure_label}] {previous_structure_name}")
                coords = get_template_coords_from_idx(nifti_label)
                if not exist_first:
                    joined_coords = np.vstack(coords)
                    exist_first = True
                else:
                    joined_coords = np.concatenate((coords, joined_coords), axis=1)
            else:
                pass
            structure_mapping, roi_hierarchy = parse_children(
                c['children'], 
                level = level + 1, 
                parent_structure = parent_structure, 
                previous_structure_label = c['acronym'], 
                previous_structure_name = c['name'],
                structure_mapping = structure_mapping,
                roi_hierarchy = roi_hierarchy,
                outdir = outdir
            )

        if create_roi:
            save_template_roi(parent_structure, previous_structure_label, joined_coords, outdir)

    return structure_mapping, roi_hierarchy

Parse the `json_file`

In [4]:
aba_onto = load_json(json_file)
main = organize_main_structures(aba_onto)
gm = organize_gm_structures(aba_onto)
wm = organize_wm_structures(aba_onto)
vs = organize_vs_structures(aba_onto)

Generate ROIs as niftis, storing outputs to `roi_dir`

In [5]:
all_label_mappings = {}
all_label_hierarchies = {}
for structure in [gm, wm, vs]:
    for structure_ix, (k, v) in enumerate(structure.items()):
        print(f"[{structure_ix + 1}/{len(structure)}] {v['name']} {v['acronym']}")
        parent_label = f"{v['name']} {v['acronym']}"
        label_mapping, label_hierarchy = parse_children(v['children'], level=1, parent_structure=v['acronym'], outdir=roi_dir)
        all_label_mappings[parent_label] = label_mapping
        all_label_hierarchies[parent_label] = label_hierarchy

[1/3] Cerebrum CH
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-FRP.nii.gz already exists.
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-MOp.nii.gz already exists.
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-MOs.nii.gz already exists.
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-SSp-n.nii.gz already exists.
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-SSp-bfd.nii.gz already exists.
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-SSp-ll.nii.gz already exists.
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-SSp-m.nii.gz already exists.
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-SSp-ul.nii.gz already exists.
/opt/animalfmritools/animalfmritools/data_template/MouseABA/rois/P56_desc-CH_roi-SSp-tr.nii.gz already e

Clean-up generated ROIs
- Remove roi-HIP (small ROI - unsure what it is)

In [5]:
!rm {roi_dir}/*roi-HIP*

Sort niftis by grey matter structures
 - isocortex **Isocortex**
 - olfactory areas **OLF**
 - hippocampal formation **HPF** (hippocampal region **HIP** / retrohippocampal region **RHP**)
 - interbrain **IB** (thalamus **TH**/hypothalamus **HY**)
 - midbrain **MB**
 - hindbrain **HB** (pons **P**/medulla **MY**)
 - cerebellar cortex **CBX**

In [6]:
def get_roi_path(roi_dir, roi_acronym, parent_acronym = 'CH'):

    roi_path = roi_dir / f"P56_desc-{parent_acronym}_roi-{roi_acronym}.nii.gz"
    if roi_path.exists():
        return roi_path

def search_for_roi_paths(roi_dir, all_label_hierarchies, main_k, sub_k, parent_acronym, roi_paths = None):
    
    all_keys = [i for i in all_label_hierarchies[main_k].keys()]

    if roi_paths is None:
        roi_paths = []
    
    for s in all_label_hierarchies[main_k][sub_k]:
        roi_path = get_roi_path(roi_dir, s, parent_acronym)
        if s not in all_keys:
            continue
            
        if roi_path is not None:
            roi_paths.append(roi_path)
            roi_paths = search_for_roi_paths(roi_dir, all_label_hierarchies, main_k, s, parent_acronym, roi_paths = roi_paths)
        else:
            roi_paths = search_for_roi_paths(roi_dir, all_label_hierarchies, main_k, s, parent_acronym, roi_paths = roi_paths)

    return roi_paths

# Cerebrum
k = "Cerebrum CH"
isocortex = search_for_roi_paths(roi_dir, all_label_hierarchies, k, "Isocortex", "CH")
olf = search_for_roi_paths(roi_dir, all_label_hierarchies, k, "OLF", "CH")
hpf = search_for_roi_paths(roi_dir, all_label_hierarchies, k, "HPF", "CH")

# Brain stem
k = 'Brain stem BS'
interbrain = search_for_roi_paths(roi_dir, all_label_hierarchies, k, "IB", "BS")
midbrain = search_for_roi_paths(roi_dir, all_label_hierarchies, k, "MB", "BS")
hindbrain = search_for_roi_paths(roi_dir, all_label_hierarchies, k, "HB", "BS")

# Cerebellum
k = 'Cerebellum CB'
cerebellar_cortex = search_for_roi_paths(roi_dir, all_label_hierarchies, k, "CBX", "CB")

Number of regions

In [7]:
print(f"ISOCORTEX: {len(isocortex)}")
print(f"OLF: {len(olf)}")
print(f"HPF: {len(hpf)}")
print(f"interbrain: {len(interbrain)}")
print(f"midbrain: {len(midbrain)}")
print(f"hindbrain: {len(hindbrain)}")
print(f"cerebellar cortex: {len(cerebellar_cortex)}")

ISOCORTEX: 43
OLF: 5
HPF: 6
interbrain: 24
midbrain: 10
hindbrain: 16
cerebellar cortex: 5


Create isocortex roi

In [8]:
def combine_rois(roi_paths, out_path):
    for ix, roi_path in enumerate(roi_paths):
        if ix == 0:
            img = nib.load(roi_path)
            data = (img.get_fdata() > 0).astype(int)
        else:
            data += (nib.load(roi_path).get_fdata() > 0).astype(int)

    data_img = nib.Nifti1Image(data, affine=img.affine, header=img.header)
    nib.save(data_img, out_path)

isocortex_path = Path(str(isocortex[0]).replace("roi-FRP", "roi-Isocortex"))
combine_rois(isocortex, isocortex_path)
assert isocortex_path.exists(), f"{isocortex_path} does not exist."

Project BOLD data from NIFTI to DTSERIES

In [9]:
bold_paths = !ls /opt/animalfmritools/animalfmritools/data/MouseAD/bids/derivatives/bold_preproc/sub-FGD3159F2/ses-20220222/func/*nii.gz
inner_surfs = !ls /opt/animalfmritools/animalfmritools/data_template/MouseABA/surfaces/*.inner.*
outer_surfs = !ls /opt/animalfmritools/animalfmritools/data_template/MouseABA/surfaces/*.outer.*

# ISOCORTEX path
volume_path = Path("/tmp") / "volume_roi_path.nii.gz"
!flirt -in {isocortex_path} -ref {bold_paths[0]} -out {volume_path} -interp nearestneighbour -applyxfm -usesqform

# BOLD path
TSNR_THRS = [0,2,4,6,8,10,12]
for TSNR_THR in TSNR_THRS:
    for ix, bold_path in enumerate(bold_paths):
        print(ix, Path(bold_path).stem)

        if ix > 0: continue

        tsnr_mask = Path(f"tsnr_mask.tsnr-thr-{TSNR_THR}.{ix}.nii.gz")
        !fslmaths {bold_path} -Tmean {Path("/tmp") / "tmean.nii.gz"}
        !fslmaths {bold_path} -Tstd {Path("/tmp") / "tstd.nii.gz"}
        !fslmaths {Path("/tmp") / "tmean.nii.gz"} -div {Path("/tmp") / "tstd.nii.gz"} -thr {TSNR_THR} -bin -mul {volume_path} {tsnr_mask}

        # Surface paths
        L_inner = inner_surfs[0]
        L_outer = outer_surfs[0]
        R_inner = inner_surfs[1]
        R_outer = outer_surfs[1]
        
        # Surface BOLD path
        L_func = Path("/tmp") / "L.bold.func.gii"
        R_func = Path("/tmp") / "R.bold.func.gii"
        bold_dtseries = Path(f"BOLD.test.tsnr-thr-{TSNR_THR}.{ix}.dtseries.nii")
        
        # Project BOLD to left hemisphere
        !wb_command -volume-to-surface-mapping \
            {bold_path} {L_outer} {L_func} \
            -ribbon-constrained {L_inner} {L_outer} \
            -volume-roi {tsnr_mask}
        
        # Project BOLD to right hemisphere
        !wb_command -volume-to-surface-mapping \
            {bold_path} {R_outer} {R_func} \
            -ribbon-constrained {R_inner} {R_outer} \
            -volume-roi {tsnr_mask}
        
        # Combine L/R func.gii to generate a CIFTI dtseries
        !wb_command -cifti-create-dense-timeseries \
            {bold_dtseries} \
            -left-metric {L_func} \
            -right-metric {R_func}

0 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-01_space-template_desc-preproc_bold.nii
1 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-02_space-template_desc-preproc_bold.nii
2 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-03_space-template_desc-preproc_bold.nii
3 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-04_space-template_desc-preproc_bold.nii
0 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-01_space-template_desc-preproc_bold.nii
1 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-02_space-template_desc-preproc_bold.nii
2 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-03_space-template_desc-preproc_bold.nii
3 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-04_space-template_desc-preproc_bold.nii
0 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-01_space-template_desc-preproc_bold.nii
1 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-02_space-template_desc-preproc_bold.nii
2 sub-FGD3159F2_ses-20220222_task-rest_dir-AP_run-03_space-template_desc-preproc_bold.nii
3 sub-FGD3